# GCAT WN18RR

### Content

* I. Initial

* II. Load Corpus

* III. Train GAT

* IV. Train Conv

* V. Evaluation

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

In [ ]:
rm -rf /content/*

# I. Initial

In [ ]:
%%bash
rm -rf GCAT
git clone https://github.com/hmthanh/GCAT.git

Cloning into 'GCAT'...
Checking out files: 100% (118/118), done.


In [ ]:
!yes | cp --i /content/GCAT/* /content/

cp: -r not specified; omitting directory '/content/GCAT/data'
cp: -r not specified; omitting directory '/content/GCAT/output'


In [ ]:
%%bash
rm -rf /content/data/
rm -rf /content/output/
rm -r /content/sample_data/

mv /content/GCAT/* /content/
rm -r /content/GCAT/

rm: cannot remove '/content/sample_data/': No such file or directory


Load Google Drive if you want to save model on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFFHsBPBnEWl4Lfsfk11ppridEJgSHgRlGLSsEmIHArNWU_mywUBBY
Mounted at /content/drive


## 1. Config

## 2. WN18RR

In [ ]:
#@title Cấu hình cài đặt WN18RR { run: "auto" }

#@markdown Siêu tham số

dataset = "WN18RR" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
cuda = True #@param {type: "boolean"}
data_folder = "./data"
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}
save_gdrive = True #@param {type: "boolean"}
epochs_gat = 3600 #@param {type: "number"}
epochs_conv = 200 #@param {type: "number"}
weight_decay_gat = float(5e-6) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = True #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = False #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 86835 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 5 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 500 #@param {type: "slider", min: 100, max: 1000, step:100}
drop_conv = 0.0  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset,data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

config

dataset: WN18RR
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: True
epochs_gat: 3600
epochs_conv: 200
weight_decay_gat: 5e-06
weight_decay_conv: 1e-05
pretrained_emb: True
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: False
batch_size_gat: 86835
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 5
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 500
drop_conv: 0.0

Load config

# II. Create corpus


In [ ]:
!python 1_create_corpus.py

number of unique_entities -> 40559
number of unique_entities -> 5173
number of unique_entities -> 5323
test_triples (3134, 3)
test_adjacency_mat (3, 3134)
entity2id 40943
relation2id 11
headTailSelector 11
unique_entities_train (40559,) 

Initialised relations and entities from TransE
Graph created
length of graph keys is  39610
time taken  2946.9735832214355
length of neighbors dict is  39115
Total triples count 93003, training triples 86835, validation_triples 3034, test_triples 3134
1. Created Corpus Successfully !


In [ ]:
!cp "/content/drive/My Drive/WN18RR_cuda_corpus.pt" "/content/data/WN18RR/WN18RR_cuda_corpus.pt"
!cp "/content/drive/My Drive/WN18RR_cuda_entity_embeddings.pt" "/content/data/WN18RR/WN18RR_cuda_entity_embeddings.pt"
!cp "/content/drive/My Drive/WN18RR_cuda_relation_embeddings.pt" "/content/data/WN18RR/WN18RR_cuda_relation_embeddings.pt"

## III. Training GAT encoder

In [ ]:
!python 2_training_encoder.py

Loading corpus
Defining model


cuda:0


# III Training conv decoder

In [ ]:
%%bash
python 3_training_decoder.py

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# V. Evaluation

### 1. Evaluating

In [ ]:
!python 4_evalution.py

Loading corpus
Loading model
Sampled indices
test set length  3134
0
4. Evaluation Successfully !


### 2. Load result

In [ ]:
ab= {}
ab[1] = "Tjanh"
ab

{1: 'Tjanh'}

In [ ]:
relation2id = ["12", "323", "234"]
left_entity_avg = {}
for i in range(len(relation2id)):
    print(i, "dsf")
    print("left_entity_avg[i]", i, " i ", left_entity_avg[i])
    print("left_entity[i]", left_entity[i])

0 dsf


KeyError: ignored

In [ ]:
left_entity_avg[0] = "234"